# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [27]:
# Libraries

import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [28]:
adm = pd.read_csv('data/Admission_Predict.csv', index_col=0)
adm

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Remove trailing spaces at the end of the column names if there are any.


In [29]:
# your code here

Let's evaluate the dataset by looking at the `head` function.

In [30]:
adm.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [31]:
adm.isna().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [32]:
adm['SN'] = adm.index
adm['SN']

Serial No.
1        1
2        2
3        3
4        4
5        5
      ... 
381    381
382    382
383    383
384    384
385    385
Name: SN, Length: 385, dtype: int64

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [33]:
adm['constructed_SN'] = adm['CGPA'].astype(str) + '_' +  adm['GRE Score'].astype(str) 
len(adm['constructed_SN'].unique()) == adm['constructed_SN'].count()

True

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [45]:
adm.set_index(['GRE Score','CGPA'], inplace=True)
adm.reset_index(inplace=True)
adm







,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,SN,constructed_SN
0,337,9.65,118,4,4.5,4.5,1,0.92,1,9.65_337
1,316,8.00,104,3,3.0,3.5,1,0.72,2,8.0_316
2,322,8.67,110,3,3.5,2.5,1,0.80,3,8.67_322
3,314,8.21,103,2,2.0,3.0,0,0.65,4,8.21_314
4,330,9.34,115,5,4.5,3.0,1,0.90,5,9.34_330
...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,110,3,3.5,3.5,1,0.82,381,9.04_324
381,325,9.11,107,3,3.0,3.5,1,0.84,382,9.11_325
382,330,9.45,116,4,5.0,4.5,1,0.91,383,9.45_330
383,312,8.78,103,3,3.5,4.0,0,0.67,384,8.78_312


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [46]:
adm[adm['CGPA']>9]

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,SN,constructed_SN
0,337,9.65,118,4,4.5,4.5,1,0.92,1,9.65_337
4,330,9.34,115,5,4.5,3.0,1,0.90,5,9.34_330
10,328,9.10,112,4,4.0,4.5,1,0.78,11,9.1_328
19,328,9.50,116,5,5.0,5.0,1,0.94,20,9.5_328
20,334,9.70,119,5,5.0,4.5,1,0.95,21,9.7_334
...,...,...,...,...,...,...,...,...,...,...
379,329,9.23,111,4,4.5,4.0,1,0.89,380,9.23_329
380,324,9.04,110,3,3.5,3.5,1,0.82,381,9.04_324
381,325,9.11,107,3,3.0,3.5,1,0.84,382,9.11_325
382,330,9.45,116,4,5.0,4.5,1,0.91,383,9.45_330


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [64]:
condition = (adm['CGPA'] > 9)&(adm['SOP']<3.5)
adm[condition]

condition.mean()


0.012987012987012988

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [55]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    return (col - np.mean(col))/np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [59]:
# your code here
CGPA_std = standardize(adm['CGPA'])
GRE_std = standardize(adm['GRE Score'])

admissions = pd.DataFrame({'CGPA_std':CGPA_std, 'GRE_std' : GRE_std})
admissions

,CGPA_std,GRE_std
0,1.750174,1.755663
1,-0.992501,-0.063450
2,0.121191,0.456297
3,-0.643433,-0.236698
4,1.234884,1.149292
...,...,...
380,0.736216,0.629546
381,0.852571,0.716170
382,1.417729,1.149292
383,0.304036,-0.409947


We will generate the decision choice at random using the code below. Please run the cell.

In [60]:
# Libraries
from random import choices

In [61]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [62]:
# your code here
adm['deciding_column'] = admissions.lookup(admissions.index, decision_choice)
adm

C:\Users\cdynm\AppData\Local\Temp\ipykernel_15532\3938242784.py:2: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  adm['deciding_column'] = admissions.lookup(admissions.index, decision_choice)


KeyError: 'One or more column labels was not found'

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
# your code here

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here